In [20]:
import numpy as np
from polyphys.manage.utils import *

In [24]:
number_density_cube(1000, 1.0, 10.0)

1.3717421124828533

In [ ]:
positions = np.array([[1.0, 1.0, 1.0], [2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])
print(end_to_end(positions))

In [ ]:
positions = np.array([[1, 0, 0], [1, 2, 2]])
transverse_size(positions, axis=0)

In [ ]:
positions = np.array([[1, 0, 0], [1, 2, 2]])
transverse_size(positions, axis=1)

In [ ]:
positions = np.array([[1, 2, 3], [4, 8, 6]])
fsd(positions, axis=1)

In [27]:
sort_filenames(['file1.data', 'file2.trj', 'file1.trj', 'file2.data'], ['data', ('lammpstrj', 'trj')])

[['file1.data', 'file2.data'], ['file2.trj', 'file1.trj']]


[('file1.data', 'file1.trj'), ('file2.data', 'file2.trj')]

In [8]:
pbc = {0: 8.0}
positions = np.array([[1.0, 2.0, 3.0], [9.5, 2.0, 3.0]])
pair_distance(positions, pbc)

array([0.5, 0. , 0. ])

In [14]:
simple_stats('energy', np.array([1.0, 2.0, 3.0]))

{'energy_mean': np.float64(2.0),
 'energy_var': np.float64(1.0),
 'energy_sem': np.float64(0.5773502691896258)}

In [16]:
sphere_sphere_intersection(3, 4, 10)

0.0

In [17]:
sphere_sphere_intersection(3, 4, 2)

94.90227807719167

In [19]:
positions= np.array([[1, 0, 0], [0, 1, 0], [0, 0, 2]])
edges = np.array([0, 1, 2, 3])
radial_histogram(positions, edges, (0, 3))

array([0, 2, 1])